In [28]:
import requests
import json
from pprint import pprint
from dotenv import load_dotenv
import os
from geopy.distance import geodesic
import json
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys


load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")
HOST= os.getenv("SQL_HOST")
USER= os.getenv("SQL_USER")
DATABASE= os.getenv("SQL_DATABASE")
PASSWORD=os.getenv("SQL_PASSWORD")

conn = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)

cursor = conn.cursor()

In [16]:

def getPlaces(latitude, longitude, api_key):
    url = "https://places.googleapis.com/v1/places:searchNearby"

    data = {
        "maxResultCount": 20,
        "rankPreference": "DISTANCE",
        "locationRestriction": {
            "circle": {
                "center": {
                    "latitude": latitude,
                    "longitude": longitude
                },
                "radius": 50000.0
            }
        }
    }

    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "places.displayName,places.location,places.formattedAddress,places.types,places.primaryType,places.id"
    }

    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()  # Will raise an HTTPError for non-200 responses

    return response.json()

In [27]:

# Define the endpoint and payload
url = "https://places.googleapis.com/v1/places:searchNearby"

data = {
    "maxResultCount": 20,
    "rankPreference": "DISTANCE",
    "locationRestriction": {
        "circle": {
            "center": {
                "latitude": 34.861743,
                "longitude": -82.40443
            },
            "radius": 50000.0
        }
    }
}

# Headers to include specific fields
headers = {
    "Content-Type": "application/json",
    "X-Goog-Api-Key": API_KEY,
    "X-Goog-FieldMask": "places.displayName,places.location,places.formattedAddress,places.types,places.primaryType,places.id"
}

# Send the request
response = requests.post(url, json=data, headers=headers)
places_data = response.json()

print(response)
print(response.text)


<Response [200]>
{
  "places": [
    {
      "name": "places/ChIJQ16cBsQxWIgR6TVPg9Wxfmk",
      "id": "ChIJQ16cBsQxWIgR6TVPg9Wxfmk",
      "types": [
        "bus_stop",
        "transit_station",
        "point_of_interest",
        "establishment"
      ],
      "formattedAddress": "Greenville, SC 29609, USA",
      "location": {
        "latitude": 34.861665599999995,
        "longitude": -82.4044155
      },
      "displayName": {
        "text": "Rutherford St & W Stone Ave",
        "languageCode": "en"
      },
      "primaryType": "bus_stop"
    },
    {
      "name": "places/ChIJFUdKCMQxWIgRgaejy3AWcdU",
      "id": "ChIJFUdKCMQxWIgRgaejy3AWcdU",
      "types": [
        "church",
        "place_of_worship",
        "point_of_interest",
        "establishment"
      ],
      "formattedAddress": "222 Rutherford St, Greenville, SC 29609, USA",
      "location": {
        "latitude": 34.8620266,
        "longitude": -82.4043009
      },
      "displayName": {
        "text": "Tr

In [4]:
pprint(places_data)

{'places': [{'displayName': {'languageCode': 'en',
                             'text': 'Rutherford St & W Stone Ave'},
             'formattedAddress': 'Greenville, SC 29609, USA',
             'location': {'latitude': 34.861665599999995,
                          'longitude': -82.4044155},
             'primaryType': 'bus_stop',
             'types': ['bus_stop',
                       'transit_station',
                       'point_of_interest',
                       'establishment']},
            {'displayName': {'languageCode': 'en',
                             'text': 'Triune Mercy Center'},
             'formattedAddress': '222 Rutherford St, Greenville, SC 29609, USA',
             'location': {'latitude': 34.8620266, 'longitude': -82.4043009},
             'primaryType': 'church',
             'types': ['church',
                       'place_of_worship',
                       'point_of_interest',
                       'establishment']},
            {'displayName': {'lang

In [11]:
#print all values
for location in places_data.get("places", []):
    address = location.get('formattedAddress', 'NULL')
    lat = location.get('location', {}).get('latitude', 'NULL')
    lon = location.get('location', {}).get('longitude', 'NULL')
    primary_type = location.get('primaryType', 'NULL')
    display_name = location.get('displayName', {}).get('text', 'NULL')

    print(f"{address}\n{lat}\n{lon}\n{primary_type}\n{display_name}\n")


Greenville, SC 29609, USA
34.861665599999995
-82.4044155
bus_stop
Rutherford St & W Stone Ave

222 Rutherford St, Greenville, SC 29609, USA
34.8620266
-82.4043009
church
Triune Mercy Center

222 Rutherford St, Greenville, SC 29609, USA
34.8620136
-82.4041982
food
Triune Mercy Center - Food Distribution Center

214 Rutherford St, Greenville, SC 29609, USA
34.8614047
-82.4040622
barbecue_restaurant
Lewis Barbecue Greenville

Greenville, SC 29609, USA
34.861143999999996
-82.404657
bus_stop
Rutherford St & Echols St

413 W Stone Ave Suite 101, Greenville, SC 29609, USA
34.8621931
-82.405002
veterinary_care
Stone Ave Vet

301 Rutherford St suite e, Greenville, SC 29609, USA
34.8621972
-82.4052147
insurance_agency
Fleming Insurance

201 Rutherford St, Greenville, SC 29609, USA
34.861001099999996
-82.40499249999999
fast_food_restaurant
Hardee’s

204 Rutherford St, Greenville, SC 29609, USA
34.86082
-82.40419
fast_food_restaurant
Arby's

307 Rutherford St, Greenville, SC 29609, USA
34.86272539

In [5]:
# Extract the list of places from the JSON
places = places_data.get("places", [])

# Filter places where the formatted address starts with a digit
filtered_places = [
    place for place in places
    if "formattedAddress" in place and place["formattedAddress"][0].isdigit()
]
for location in filtered_places:
    address = location.get('formattedAddress', 'NULL')
    lat = location.get('location', {}).get('latitude', 'NULL')
    lon = location.get('location', {}).get('longitude', 'NULL')
    primary_type = location.get('primaryType', 'NULL')
    display_name = location.get('displayName', {}).get('text', 'NULL')

    print(f"{address}\n{lat}\n{lon}\n{primary_type}\n{display_name}\n")


222 Rutherford St, Greenville, SC 29609, USA
34.8620266
-82.4043009
church
Triune Mercy Center

222 Rutherford St, Greenville, SC 29609, USA
34.8620136
-82.4041982
food
Triune Mercy Center - Food Distribution Center

214 Rutherford St, Greenville, SC 29609, USA
34.8614047
-82.4040622
barbecue_restaurant
Lewis Barbecue Greenville

413 W Stone Ave Suite 101, Greenville, SC 29609, USA
34.8621931
-82.405002
veterinary_care
Stone Ave Vet

301 Rutherford St suite e, Greenville, SC 29609, USA
34.8621972
-82.4052147
insurance_agency
Fleming Insurance

201 Rutherford St, Greenville, SC 29609, USA
34.861001099999996
-82.40499249999999
fast_food_restaurant
Hardee’s

204 Rutherford St, Greenville, SC 29609, USA
34.86082
-82.40419
fast_food_restaurant
Arby's

307 Rutherford St, Greenville, SC 29609, USA
34.862725399999995
-82.4048583
chiropractor
The Cagle Center Of Wellness

329 W Stone Ave, Greenville, SC 29609, USA
34.862031
-82.4031901
NULL
Giraffe Web Development & Design | Greenville SC

413 

In [26]:
print(places_data.get("places", [])[1].get('displayName', {}).get('text', 'NULL'))
print(places_data.get("places", [])[2].get('displayName', {}).get('text', 'NULL'))

Triune Mercy Center
Triune Mercy Center - Food Distribution Center


In [22]:
#get furthest point (radius of area)
reference_point = (34.861743, -82.40443)

# Assume places_data is already defined from your Places API v1 response
location_array = []
for i in places_data['places']:
    lat = i['location']['latitude']
    lng = i['location']['longitude']
    location_array.append((lat, lng))

# Calculate distances from reference point
distances = [
    (geodesic(reference_point, loc).meters, loc)
    for loc in location_array
]

# Find the furthest point
max_distance, furthest_point = max(distances, key=lambda x: x[0])

print(f"Furthest point: {furthest_point}")
print(f"Distance: {max_distance:} meters")

Furthest point: (34.860597299999995, -82.405853)
Distance: 181.8972281172954 meters


In [11]:
#define the area of the circle created in places_data
def get_max_distance(lat, lon, places_data):
    reference_point = (lat, lon)
    
    distances = [
        geodesic(reference_point, (place['location']['latitude'], place['location']['longitude'])).meters
        for place in places_data['places']
    ]
    
    return max(distances) if distances else 0

181.8972281172954


In [10]:
#add websites to places
def get_websites(filtered_places)
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 2)
    #for location in places_data.get("places", []):
    for location in filtered_places:
        # Open the target URL
        driver.get("https://www.google.com/maps/place/")
        
        businessAddress=location.get('formattedAddress', 'NULL')
        
        print(businessAddress)
        # Clear any pre-filled text and enter your query
        if not businessAddress[0].isdigit():
            print("not an address")
            continue
            
    
        search_input = wait.until(EC.visibility_of_element_located((By.ID, "searchboxinput")))
    
        search_input.send_keys(businessAddress)
        
        search_input.send_keys(Keys.ENTER)
    
        businessName=location.get('displayName', {}).get('text', 'NULL')
        xpath_selector = f"//button[@aria-label='{businessName}']"
        
        
        # Wait for button to be clickable
        try:
            button = wait.until(EC.element_to_be_clickable((By.XPATH, xpath_selector)))
            button.click()
        except:
            search_input.clear()
            search_input.send_keys(businessName)
            search_input.send_keys(Keys.ENTER)
            """
            print("no button click")
            continue
            """
        
        # Click the button
    
        
        
        # Locate the element using the 'data-tooltip' attribute
        try:
            element = wait.until(
                EC.element_to_be_clickable((
                    By.XPATH,
                    "//a[contains(@class, 'CsEnBe') and @data-tooltip='Open website']"
                ))
            )
        except:
            print("no website")
            location['website']=None
            continue
        
        
        # Get the href attribute
        href_value = element.get_attribute('href')
        
        # Print the href value
        #print("Website URL:", href_value)
        location['website']=href_value
    #Close the driver
    driver.quit()
    return filtered_places

222 Rutherford St, Greenville, SC 29609, USA
Website URL: http://triunemercy.org/
222 Rutherford St, Greenville, SC 29609, USA
Website URL: https://www.triunemercy.org/food.html
214 Rutherford St, Greenville, SC 29609, USA
Website URL: https://www.lewisbarbecue.com/
413 W Stone Ave Suite 101, Greenville, SC 29609, USA
Website URL: http://www.stoneavevet.com/
301 Rutherford St suite e, Greenville, SC 29609, USA
Website URL: https://www.fleminginsurancesc.com/
201 Rutherford St, Greenville, SC 29609, USA
Website URL: https://locations.hardees.com/sc/greenville/201-rutherford-street
204 Rutherford St, Greenville, SC 29609, USA
no website
307 Rutherford St, Greenville, SC 29609, USA
Website URL: http://www.caglecenter.com/
329 W Stone Ave, Greenville, SC 29609, USA
Website URL: https://www.giraffeweb.net/
413 W Stone Ave #102, Greenville, SC 29609, USA
Website URL: https://www.carolinaanimaleye.com/
332 W Stone Ave, Greenville, SC 29609, USA
no website
323 W Stone Ave, Greenville, SC 29609

In [13]:
print(filtered_places[6])

{'types': ['fast_food_restaurant', 'sandwich_shop', 'deli', 'meal_takeaway', 'american_restaurant', 'food_store', 'store', 'restaurant', 'food', 'point_of_interest', 'establishment'], 'formattedAddress': '204 Rutherford St, Greenville, SC 29609, USA', 'location': {'latitude': 34.86082, 'longitude': -82.40419}, 'displayName': {'text': "Arby's", 'languageCode': 'en'}, 'primaryType': 'fast_food_restaurant', 'website': None}


In [19]:

# -------------------------------
# STEP 2: Setup Selenium
# -------------------------------

options = webdriver.ChromeOptions()
# options.add_argument("--headless")  # Uncomment if you want headless mode
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 4)

# -------------------------------
# STEP 3: Scrape Website URLs
# -------------------------------

for location in filtered_places:
    business_address = location.get("formattedAddress", "")
    business_name = location.get("displayName", {}).get("text", "")

    print(f"Searching: {business_name} - {business_address}")

    try:
        driver.get("https://www.google.com/maps/place/")
        
        # Wait for search box and enter the address
        search_input = wait.until(EC.visibility_of_element_located((By.ID, "searchboxinput")))
        search_input.clear()
        search_input.send_keys(business_address)
        search_input.send_keys(Keys.ENTER)

        # Wait for results to load

        # Try to find website link
        try:
            website_button = wait.until(EC.element_to_be_clickable((
                By.XPATH,
                "//a[contains(@class, 'CsEnBe') and @data-tooltip='Open website']"
            )))
            website_url = website_button.get_attribute("href")
            location["website"] = website_url
            print("✔ Website found:", website_url)
        except:
            location["website"] = None
            print("✖ No website found")

    except Exception as e:
        location["website"] = None
        print(f"⚠ Error searching for {business_address}: {e}")



# -------------------------------
# STEP 5: Close Selenium
# -------------------------------

driver.quit()


Searching: Triune Mercy Center - 222 Rutherford St, Greenville, SC 29609, USA
✖ No website found
Searching: Triune Mercy Center - Food Distribution Center - 222 Rutherford St, Greenville, SC 29609, USA
✖ No website found
Searching: Lewis Barbecue Greenville - 214 Rutherford St, Greenville, SC 29609, USA
✖ No website found
Searching: Stone Ave Vet - 413 W Stone Ave Suite 101, Greenville, SC 29609, USA
✖ No website found
Searching: Fleming Insurance - 301 Rutherford St suite e, Greenville, SC 29609, USA
✖ No website found
Searching: Hardee’s - 201 Rutherford St, Greenville, SC 29609, USA
✖ No website found
Searching: Arby's - 204 Rutherford St, Greenville, SC 29609, USA
✖ No website found
Searching: The Cagle Center Of Wellness - 307 Rutherford St, Greenville, SC 29609, USA
✖ No website found
Searching: Giraffe Web Development & Design | Greenville SC - 329 W Stone Ave, Greenville, SC 29609, USA
✖ No website found
Searching: Carolina Animal Eye - 413 W Stone Ave #102, Greenville, SC 2960

In [ ]:
def collect_places_data(current_circle_number):
    #get the current area of the circle
    current_area=calculate_geojson_area(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
    #get the calculated maximum area of the circle
    with open(r"C:\Users\Joe\Desktop\geojsons\geojson_total_areas.json") as f:
        areas = json.load(f)
    max_area=areas[f"circle{current_circle_number}.geojson"]

    #loop until 97% of circle has been mapped
    while (current_area/max_area)>0.03:
        #generate a random point that has not been mapped
        randomPoint=get_random_point_within_geojson(rf"C:\Users\Joe\Desktop\geojsons\circle{current_circle_number}.geojson")
        #get google places data from that random point
        places_data=getPlaces(randomPoint[0],randomPoint[1], API_KEY)
        #get the size of the circle created from the google places data
        places_radius=get_max_distance(randomPoint[0],randomPoint[1], places_data)
        #filter the places data for non-businesses
        filtered_places=[
        place for place in places_data.get("places", [])
        if "formattedAddress" in place and place["formattedAddress"][0].isdigit()]
        #use selenium to add website to places_data
        filtered_places=get_websites(filtered_places)
        #add places to database
        
        #modify geojson to remove the area 

In [34]:
#create initial database
conn = mysql.connector.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    database=DATABASE
)

cursor = conn.cursor()

create_table_query = """
CREATE TABLE IF NOT EXISTS businesses (
    business_id INT AUTO_INCREMENT PRIMARY KEY,
    business_name VARCHAR(255),
    primary_type VARCHAR(255),
    website VARCHAR(255),
    business_address VARCHAR(255),
    coordinates POINT NOT NULL,
    google_id VARCHAR(255),
    SPATIAL INDEX(coordinates)
);
"""
create_business_types_query="""
CREATE TABLE IF NOT EXISTS business_types (
    id INT AUTO_INCREMENT PRIMARY KEY,
    business_id INT,
    type VARCHAR(100),
    FOREIGN KEY (business_id) REFERENCES businesses(business_id)
);
"""
cursor.execute(create_table_query)
conn.commit()
cursor.execute(create_business_types_query)
conn.commit()

cursor.close()
conn.close()